# Use Socrata API to pull some other open Chicago data, starting with L stop locations


# This has been implemented in the library, but i'll leave the notebook in case I want to add functionality

In [15]:
import os
import sys

import numpy as np
import pandas as pd

import folium
import matplotlib.pyplot as plt

%matplotlib inline

# Load from lib
%load_ext autoreload 
%autoreload 2

# Add library to path 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from lib.download_data import load_trip_data, load_station_data, query_cta_stations
from lib.visualization import map_stations

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
df_cta = query_cta_stations()

In [17]:
df_cta.head()

,ada,blue,brn,direction_id,g,location,map_id,o,p,pexp,pnk,red,station_descriptive_name,station_name,stop_id,stop_name,y,latitude,longitude
0,True,False,False,W,False,"{'type': 'Point', 'coordinates': [-87.669147, ...",40830,False,False,False,True,False,18th (Pink Line),18th,30162,18th (54th/Cermak-bound),False,41.857908,-87.669147
1,True,False,False,E,False,"{'type': 'Point', 'coordinates': [-87.669147, ...",40830,False,False,False,True,False,18th (Pink Line),18th,30161,18th (Loop-bound),False,41.857908,-87.669147
2,True,False,False,N,False,"{'type': 'Point', 'coordinates': [-87.680622, ...",40120,True,False,False,False,False,35th/Archer (Orange Line),35th/Archer,30022,35th/Archer (Loop-bound),False,41.829353,-87.680622
3,True,False,False,S,False,"{'type': 'Point', 'coordinates': [-87.680622, ...",40120,True,False,False,False,False,35th/Archer (Orange Line),35th/Archer,30023,35th/Archer (Midway-bound),False,41.829353,-87.680622
4,True,False,False,S,True,"{'type': 'Point', 'coordinates': [-87.625826, ...",41120,False,False,False,False,False,35th-Bronzeville-IIT (Green Line),35th-Bronzeville-IIT,30214,35-Bronzeville-IIT (63rd-bound),False,41.831677,-87.625826


In [41]:
df_stations = load_station_data()

In [42]:
df_stations.head()

,id,name,city,latitude,longitude,dpcapacity,online_date,Unnamed: 7,online_datetime,online_month,online_year,distance_to_closest_L_station
0,2,Buckingham Fountain,Chicago,41.876393,-87.620328,27,2013-06-10,NaN,2013-06-10 10:43:00,2013-06,2013,0.006503
1,3,Shedd Aquarium,Chicago,41.867226,-87.615355,55,2013-06-10,NaN,2013-06-10 10:44:00,2013-06,2013,0.011236
2,4,Burnham Harbor,Chicago,41.856268,-87.613348,23,2013-06-10,NaN,2013-06-10 10:46:00,2013-06,2013,0.013429
3,5,State St & Harrison St,Chicago,41.874053,-87.627716,23,2013-06-10,NaN,2013-06-10 10:46:00,2013-06,2013,0.000237
4,6,Dusable Harbor,Chicago,41.885041,-87.612794,39,2013-06-10,NaN,2013-06-10 11:18:00,2013-06,2013,0.013518


In [31]:
# Distance function
def eucl_d(lat1, lat2, long1, long2):
    return np.sqrt((lat2-lat1)**2 + (long2-long1)**2)
eucl_d(1,2,3,4)

1.4142135623730951

In [38]:
df_stations['distance_to_closest_L_station'] = df_stations.apply(lambda x: min(eucl_d(x.latitude, df_cta.latitude, x.longitude, df_cta.longitude)), axis=1)

In [39]:
df_stations.head()

,id,name,city,latitude,longitude,dpcapacity,online_date,Unnamed: 7,online_datetime,online_month,online_year,distance_to_closest_L_station
0,2,Buckingham Fountain,Chicago,41.876393,-87.620328,27,2013-06-10,NaN,2013-06-10 10:43:00,2013-06,2013,0.006503
1,3,Shedd Aquarium,Chicago,41.867226,-87.615355,55,2013-06-10,NaN,2013-06-10 10:44:00,2013-06,2013,0.011236
2,4,Burnham Harbor,Chicago,41.856268,-87.613348,23,2013-06-10,NaN,2013-06-10 10:46:00,2013-06,2013,0.013429
3,5,State St & Harrison St,Chicago,41.874053,-87.627716,23,2013-06-10,NaN,2013-06-10 10:46:00,2013-06,2013,0.000237
4,6,Dusable Harbor,Chicago,41.885041,-87.612794,39,2013-06-10,NaN,2013-06-10 11:18:00,2013-06,2013,0.013518


In [40]:
df_stations.sort_values('distance_to_closest_L_station')

,id,name,city,latitude,longitude,dpcapacity,online_date,Unnamed: 7,online_datetime,online_month,online_year,distance_to_closest_L_station
59,66,Clinton St & Lake St,Chicago,41.885637,-87.641823,23,2013-06-25,NaN,2013-06-25 12:21:00,2013-06,2013,0.000058
40,44,State St & Randolph St,Chicago,41.884730,-87.627734,27,2013-06-25,NaN,2013-06-25 10:59:00,2013-06,2013,0.000112
570,611,Oak Park Ave & South Blvd,Oak Park,41.886923,-87.793899,19,2016-06-23,NaN,2016-06-23 12:25:00,2016-06,2016,0.000133
415,451,Sheridan Rd & Loyola Ave,Chicago,42.001044,-87.661198,15,2015-02-10,NaN,2015-02-10 14:03:00,2015-02,2015,0.000140
63,71,Morgan St & Lake St,Chicago,41.885483,-87.652305,23,2013-06-25,NaN,2013-06-25 12:55:00,2013-06,2013,0.000152
182,201,Indiana Ave & 40th St,Chicago,41.821680,-87.621600,15,2013-08-10,NaN,2013-08-10 15:47:00,2013-08,2013,0.000235
3,5,State St & Harrison St,Chicago,41.874053,-87.627716,23,2013-06-10,NaN,2013-06-10 10:46:00,2013-06,2013,0.000237
347,378,California Ave & Lake St,Chicago,41.884454,-87.696298,15,2015-01-30,NaN,2015-01-30 14:00:00,2015-01,2015,0.000243
56,60,Dayton St & North Ave,Chicago,41.910578,-87.649422,19,2013-06-25,NaN,2013-06-25 12:08:00,2013-06,2013,0.000257
34,38,Clark St & Lake St,Chicago,41.886021,-87.630876,27,2013-06-25,NaN,2013-06-25 01:43:00,2013-06,2013,0.000284


In [5]:
import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofchicago.org", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofchicago.org,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("8mj8-j3c4", limit=2000)

# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

In [6]:
df.head()

,ada,blue,brn,direction_id,g,location,map_id,o,p,pexp,pnk,red,station_descriptive_name,station_name,stop_id,stop_name,y
0,True,False,False,W,False,"{'type': 'Point', 'coordinates': [-87.669147, ...",40830,False,False,False,True,False,18th (Pink Line),18th,30162,18th (54th/Cermak-bound),False
1,True,False,False,E,False,"{'type': 'Point', 'coordinates': [-87.669147, ...",40830,False,False,False,True,False,18th (Pink Line),18th,30161,18th (Loop-bound),False
2,True,False,False,N,False,"{'type': 'Point', 'coordinates': [-87.680622, ...",40120,True,False,False,False,False,35th/Archer (Orange Line),35th/Archer,30022,35th/Archer (Loop-bound),False
3,True,False,False,S,False,"{'type': 'Point', 'coordinates': [-87.680622, ...",40120,True,False,False,False,False,35th/Archer (Orange Line),35th/Archer,30023,35th/Archer (Midway-bound),False
4,True,False,False,S,True,"{'type': 'Point', 'coordinates': [-87.625826, ...",41120,False,False,False,False,False,35th-Bronzeville-IIT (Green Line),35th-Bronzeville-IIT,30214,35-Bronzeville-IIT (63rd-bound),False


In [9]:
df.location[0]['coordinates'][0]

-87.669147

In [12]:
df['latitude'] = df.location.map(lambda x: x['coordinates'][1])
df['longitude'] = df.location.map(lambda x: x['coordinates'][0])

In [13]:
df.head()

,ada,blue,brn,direction_id,g,location,map_id,o,p,pexp,pnk,red,station_descriptive_name,station_name,stop_id,stop_name,y,latitude,longitude
0,True,False,False,W,False,"{'type': 'Point', 'coordinates': [-87.669147, ...",40830,False,False,False,True,False,18th (Pink Line),18th,30162,18th (54th/Cermak-bound),False,41.857908,-87.669147
1,True,False,False,E,False,"{'type': 'Point', 'coordinates': [-87.669147, ...",40830,False,False,False,True,False,18th (Pink Line),18th,30161,18th (Loop-bound),False,41.857908,-87.669147
2,True,False,False,N,False,"{'type': 'Point', 'coordinates': [-87.680622, ...",40120,True,False,False,False,False,35th/Archer (Orange Line),35th/Archer,30022,35th/Archer (Loop-bound),False,41.829353,-87.680622
3,True,False,False,S,False,"{'type': 'Point', 'coordinates': [-87.680622, ...",40120,True,False,False,False,False,35th/Archer (Orange Line),35th/Archer,30023,35th/Archer (Midway-bound),False,41.829353,-87.680622
4,True,False,False,S,True,"{'type': 'Point', 'coordinates': [-87.625826, ...",41120,False,False,False,False,False,35th-Bronzeville-IIT (Green Line),35th-Bronzeville-IIT,30214,35-Bronzeville-IIT (63rd-bound),False,41.831677,-87.625826
